#### Introduction
In order to maximise the number of records, baseline values have been predicted
using either a stage 1 gam or an exponentially weighted mean (ewm). Now all survey groups 
with baseline indices (actual or predicted) need to be identified and grouped for the 
stage 2 gam analysis. All sites without a baseline must be removed from the dataset.

#### Aim
To aggregate all records with baseline indices in preperation for stage 2 GAM 
analysis. 

#### Workflow
1) GAM predicted baseline dataframes are cleaned to make format consistent with ukbms dataframe
2) ewm predicted baseline dataframes are cleaned to make format consistent with ukbms dataframe
3) ukbms dataframe (actual survey data) is filtered by removing site/species 
combinations without an actual or predicted baseline.
4) GAM and ewm predicted baselines are added to the main ukbms dataframe. 
5) log index ratios are computed for each record (these will be imputed into GAM stage 
2 models).
6) A quality check is performed to ensure the number of records in the aggregated
dataframe is correct.
7) A final filter is performed, removing species groups with more than 15% zero 
values.

In [18]:
# Importing relevant packages and datasets
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Importing localised file directory
project_root = Path(os.environ['butterfly_project'])

# Importing the main dataset (all study records)
ukbms = pd.read_csv(project_root/'Data'/'UKBMS'/'ukbms_master_v1.csv', index_col=0)

# Importing dataset with site/species combinations (merging consecutive survey groups).
gam_prediction_records = pd.read_csv(project_root/'Data'/'UKBMS'/'gam_1'/'gam_1_accept.csv', index_col=0)

# Importing GAM predictions
gam_6_7_8 = pd.read_csv(project_root/'Data'/'UKBMS'/'gam_1'/'obs_6_7_8'/'obs_6_7_8_predictions.csv', index_col=0)
gam_9_10_11 = pd.read_csv(project_root/'Data'/'UKBMS'/'gam_1'/'obs_9_10_11'/'obs_9_10_11_predictions.csv', index_col=0)
gam_12_13_14 = pd.read_csv(project_root/'Data'/'UKBMS'/'gam_1'/'obs_12_13_14'/'obs_12_13_14_predictions.csv', index_col=0)
gam_15_16_17 = pd.read_csv(project_root/'Data'/'UKBMS'/'gam_1'/'obs_15_16_17'/'obs_15_16_17_predictions.csv', index_col=0)
gam_18_19_20 = pd.read_csv(project_root/'Data'/'UKBMS'/'gam_1'/'obs_18_19_20'/'obs_18_19_20_predictions.csv', index_col=0)

# Importing ewm predictions
ewm = pd.read_csv(project_root/'Data'/'UKBMS'/'ewm'/'ewm_predictions.csv', index_col=0)

# Importing records which have no baseline and could not be predicted
indices_rejected = pd.read_csv(project_root/'Data'/'UKBMS'/'ewm'/'ewm_reject.csv', index_col=0)

In [19]:
# Removing redundant columns
ukbms = ukbms.drop(columns=['country',
                            'site_name',
                            'gridreference',
                            'easting',
                            'northing',
                            'species',
                            'common_name'])

#### Cleaning GAM Predictions

In [20]:
# Storing GAM dataframes in a list
gam_list = [gam_6_7_8, 
            gam_9_10_11, 
            gam_12_13_14, 
            gam_15_16_17, 
            gam_18_19_20]

# Formatting of prediction dataframes needs to be tidied so they are compatible with 
# main 'ukbms' dataframe. 
for i in gam_list: # GAM dataframes are accessed by looping through 'gam_list'
    i['year']=1993 # Adding a year column to all GAM dataframes
    # Back-transforming log indices to count data. Count predictions were created 
    # using np.log1p(). To back-transform np.expm1() is used.
    i['site_index'] = np.expm1(i['prediction_data'])

# All GAM predictions are referenced by 'consecutive_survey_group'. To access the
# the associated site and species codes, 'consecutive_survey_group' is used to merge
# GAM dtaframes with 'gam_predictions_records'.  
for i in np.arange(0,5,1): # Loops through items in 'gam_list'
    gam_merge = (
        gam_list[i]
        .merge(gam_prediction_records[['site_code',
                                       'species_code',
                                       'consecutive_survey_group']],
               on=['consecutive_survey_group'], # The primary key used to access codes
               how='left')
    )
    # Columns in GAM dataframes are renamed for consistency with 'ukbms' df format
    gam_merge = (
        gam_merge
        .rename(columns={'prediction_data':'log_site_index'})
    )
    # Cleaned dataframes are reassigned to 'gam_list'
    gam_list[i] = (
        gam_merge
        .drop_duplicates()
        .reset_index(drop=True)
        .drop(columns=['edof', 'consecutive_survey_group']) # Redundant columns removed
    )

#### Cleaning ewm Predictions

In [21]:
# A year column is added to the ewm (exponentially weighted mean) predictions table. 
ewm['year'] = 1993

# Redundant columns are removed. 
ewm = (
    ewm
    .rename(columns={'log_ewm':'log_site_index',
                          'ewm':'site_index'})
    .drop(columns=['record_count'])
)

#### Filtering the Main UKBMS DataFrame: Removing site/species Groups Without a Baseline Record

Now the main dataframe 'ukbms' needs to be filtered to remove all records without a 
baseline. A filter is applied removing all site/species combinations without a survey 
between 1983 and 2003 (it was not possible to predict a baseline index for 
site/species combinations without a survey during this period).

In [ ]:
ukbms_baseline_review = (
    ukbms[(ukbms['year']>=1983) & (ukbms['year']<=2003)]
    .drop_duplicates()
    .reset_index(drop=True)
)

See cell below:

Removing site/species groups that were reviewed but failed to meet the GAM or ewm criteria.

In [23]:
ukbms_indicator = (
    ukbms_baseline_review
    .merge(indices_rejected,
           on=['site_code', 'species_code'],
           how='left',
           # 'indicator' creates a new column labelling rows that appear in left df only
           indicator=True) 
)

In [24]:
# The indicator column ('_merge') is used for filtering. Only the records that appear 
# in the left dataframe are retained.
ukbms_site_species_accepted = (
    ukbms_indicator[ukbms_indicator['_merge']=='left_only'][['site_code',
                                                             'species_code']]
    .drop_duplicates() # Only unique site/species combinations are required. 
)

#### Adding a 'Buffer' Period for Improved GAM Boundary Stability
The study period is 1993-2023, but to improve GAM boundary stability, records in the 5 years
preceding the study period will be imputed into the model. 'ukbms_site_species_accepted' contains
records of all accepted site/species combinations. It is used to access all the corresponding
records from 'ukbms'.

In [25]:
ukbms_filtered = (
    ukbms.merge(ukbms_site_species_accepted,
                how='inner',
                on=['site_code','species_code'])
)

# Keeping only years to be used for model input data
ukbms_filtered = (
    ukbms_filtered[ukbms_filtered['year']>=1988] 
    .reset_index(drop=True)
)

In [ ]:
# Converting site indices to natural logs using np.log1p().
# This ensures consistency with GAM and ewm dataframes
ukbms_filtered['log_site_index'] = np.log1p(ukbms_filtered['site_index'])

#### Joining All Accepted Records: EWM Baseline Predictions, GAM baseline Predictions and All Remaining Records from 'ukbms_filtered'

In [27]:
ukbms_combined = pd.concat([ukbms_filtered, # actual records
                            ewm,
                            gam_list[0], # GAM indices accessed by indexing 'gam_list'
                            gam_list[1],
                            gam_list[2],
                            gam_list[3],
                            gam_list[4]],
                           ignore_index=True) # prevents duplicated df index labels

#### Computing Log Index Ratios for Each Record
All records in 'ukbms_combined' will be used in the analysis. Now the log index ratio can be 
calculated for each row. This is the main metric to be used in the analysis.

In [28]:
# First the baseline index is isolated 
baseline_indices = (
    ukbms_combined[ukbms_combined['year']==1993][
    ['site_code',
     'species_code',
     'log_site_index'] # the baseline index for each site/species combination
    ]
)

# Next, each site/species record is joined to its corresponding baseline.
log_baseline = (
    ukbms_combined.merge(baseline_indices,
                         on=['site_code','species_code'],
                         how='left',
                         suffixes=['', '_baseline']) # to distinguish redundant columns
)

# 'log_site_index_baseline' column is isolated by subsetting the 'log_baseline' df and
# assigned to a new variable: 'log_baseline_grouped'.
log_baseline_grouped = (
    log_baseline[['log_site_index_baseline']] 
    # The column name is modified to a more succinct heading
    .rename(columns={'log_site_index_baseline':'log_baseline'})
)

# Single column dataframe (with log baselines indices) is copied over to main dataframe 
ukbms_combined['log_baseline'] = log_baseline_grouped

# Computation of the log index ratio for each row
# Index ratio in logs, is just a subraction
ukbms_combined['log_index_ratio'] = (ukbms_combined['log_site_index'] 
                                     - ukbms_combined['log_baseline'])

#### Quality Check
Checking to see if the aggregated total number of unique site/species baseline combinations 
in 'ukbms_combined' is correct.

In [29]:
# The total number of unique records without a baseline between 1983 and 2023
# An attempt to impute missing baselines was made for all site/species combinations 
# falling within this period.
print(
    'unique site/species combinations (1983-2003) TOTAL: ' 
    + str(
        len(
            # The evaluation period for site/species combinations without a baseline
            ukbms[(ukbms['year']>=1983) & (ukbms['year']<=2003)] 
            .drop_duplicates(['site_code','species_code'])
        )
    )
)

# The aggregated total number of predicted baseline indices
print(
    'predicted baselines :' 
    + str(
        len(ewm) 
          + len(gam_list[0]) 
          + len(gam_list[1]) 
          + len(gam_list[2]) 
          + len(gam_list[3]) 
          + len(gam_list[4])
    )
)

# Actual baseline values (The sites which were surveyed in 1993)
print(
    'actual baselines :' 
    + str(
        len(ukbms[ukbms['year']==1993]
              .drop_duplicates(['site_code','species_code']))
    )
)

# The total number of baseline indices that could not be predicted
print('baselines not predicted: ' + str(len(indices_rejected)), '\n')

# Checking to see if aggregated site/species baseline combinations matches original
# number of records.
print(
    'ukbms_combined TOTAL: ' # unique site/species that will be used in the analysis
    + str(
        len(ukbms_combined[ukbms_combined['year']==1993])
    )
)

unique site/species combinations (1983-2003) TOTAL: 21027
predicted baselines :5410
actual baselines :7502
baselines not predicted: 8115 

ukbms_combined TOTAL: 12912


#### Removing Species Groups with More than 15% Zero Values

In [30]:
ukbms_combined['proportion_zeroes'] = (
    ukbms_combined
    .groupby(['species_code'])['site_index']
    # proportion of zeroes is computed as a decimal. 
    .transform(lambda x: sum(x==0)/len(x)) 
)

# Filtering to only species groups that fall below zero threshold
ukbms_zero_limit = ukbms_combined[ukbms_combined['proportion_zeroes']<=0.15]

In [31]:
# Dropping redundant column
ukbms_species_analysis = (
    ukbms_zero_limit
    .drop(columns=['proportion_zeroes'])
    .reset_index(drop=True) # Index is reset after zero filter
)

In [16]:
# Creating csv file for records accepted for analysis. 
ukbms_species_analysis.to_csv(project_root/'Data'/'UKBMS'/'gam_2'/'ukbms_species_analysis.csv')